## Setup

Run the setup and imports for the notebook.

In [1]:
import init_load_board
exec(open('init_load_board.py').read())

import init_simple_mdp
exec(open('init_simple_mdp.py').read())

import numpy as np

import function_board as fb 
import function_tool as ft
import function_get_aiming_grid 
exec(open('function_get_aiming_grid.py').read())

import evaluate_score_probability_with_error as esp
import function_solve_dp

#%%
data_parameter_dir = fb.data_parameter_dir
result_dir = './result'       


a_throw_list = []
a_token_list = []

for a in actions:
    a_throw_list.append(a)

for a in token_actions:
    a_token_list.append(a)

max_tokens = 1

name_pa = 'player{}'.format(10)


import os
import sys
import time

import function_board as fb
import function_tool as ft
import function_get_aiming_grid
import function_evaluate_policy as fep

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import torch
torch.set_printoptions(precision=4)
torch.set_printoptions(linewidth=300)
torch.set_printoptions(threshold=300)

playerID_list = [10]
function_get_aiming_grid.save_aiming_grid_custom_no_tokens(playerID_list)
function_get_aiming_grid.save_aiming_grid_custom_tokens(playerID_list)
function_get_aiming_grid.save_aiming_grid_custom_only_tokens(playerID_list)

[aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore, prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

import function_solve_dp_tokens as fsdt

prob_grid_doublescore_dic = None
prob_grid_doublescore_dic_t = None

generate and save action set grid_version=custom_no_tokens
load_pickle from ./data_parameter/player_gaussin_fit/grid_full/player10_gaussin_prob_grid.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_full/playert_gaussin_prob_grid.pkl
dump_pickle to ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl

generate and save action set grid_version=custom_tokens
load_pickle from ./data_parameter/player_gaussin_fit/grid_full/player10_gaussin_prob_grid.pkl
dump_pickle to ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl

generate and save action set grid_version=custom_only_tokens
load_pickle from ./data_parameter/player_gaussin_fit/grid_full/playert_gaussin_prob_grid.pkl
dump_pickle to ./data_parameter/player_gaussin_fit/grid_custom_only_tokens/player10_gaussin_prob_grid_custom_only_tokens.pkl

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_ga

## Token Code

Set the number of tokens in the first cell and then run the second cell to solve.

In [2]:
tokens = 9
max_token_index = tokens + 1

In [3]:
num_aiming_location = aiming_grid.shape[0]
prob_normalscore_1tosmax_dic = {}
prob_normalscore_1tosmaxsum_dic = {}
prob_normalscore_1tosmax_dic_t = {}
prob_normalscore_1tosmaxsum_dic_t = {}

# No tokens
for score_max in range(0,61):
    score_max_plus1 = score_max + 1 
    prob_normalscore_1tosmax_dic[score_max] = np.array(prob_grid_normalscore[:,1:score_max_plus1])
    prob_normalscore_1tosmaxsum_dic[score_max] = prob_normalscore_1tosmax_dic[score_max].sum(axis=1)
if prob_grid_doublescore_dic is None:
    prob_doublescore_dic = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
else:
    prob_doublescore_dic = prob_grid_doublescore_dic
prob_DB = np.array(prob_grid_bullscore[:,1])

# Tokens
for score_max in range(0,61):
    score_max_plus1 = score_max + 1 
    prob_normalscore_1tosmax_dic_t[score_max] = np.array(prob_grid_normalscore_t[:,1:score_max_plus1])
    prob_normalscore_1tosmaxsum_dic_t[score_max] = prob_normalscore_1tosmax_dic_t[score_max].sum(axis=1)
if prob_grid_doublescore_dic_t is None:
    prob_doublescore_dic_t = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic_t[doublescore] = np.array(prob_grid_doublescore_t[:,doublescore_index])
else:
    prob_doublescore_dic_t = prob_grid_doublescore_dic_t
prob_DB_t = np.array(prob_grid_bullscore_t[:,1])

## possible state: s = 0,1(not possible),2,...,501
optimal_value = np.zeros((max_token_index,502))
#optimal_value[1] = np.nan
optimal_action_index = np.zeros((max_token_index,502), np.int32)
optimal_action_index[:,0] = -1
optimal_action_index[:,1] = -1

for t in range(0,max_token_index):

    if t==0: 

        for score_state in range(2,502):            
            ## use matrix operation to search all aiming locations
            
            ## transit to less score state    
            ## s1 = min(score_state-2, 60)
            ## p[z=1]*v[score_state-1] + p[z=2]*v[score_state-2] + ... + p[z=s1]*v[score_state-s1]
            score_max = min(score_state-2, 60)
            score_max_plus1 = score_max + 1 

            ## transit to next state
            num_tothrow = 1.0 + prob_normalscore_1tosmax_dic[score_max].dot(optimal_value[0,score_state-1:score_state-score_max-1:-1])
            ## probability of transition to state other than s itself
            prob_otherstate = prob_normalscore_1tosmaxsum_dic[score_max]
            
            ## transit to the end of game
            if (score_state == fb.score_DB): ## hit double bull
                prob_otherstate += prob_DB
            elif (score_state <= 40 and score_state%2==0): ## hit double
                prob_otherstate += prob_doublescore_dic[score_state]
            else: ## game does not end
                pass
            
            ## expected number of throw for all aiming locations
            prob_otherstate = np.maximum(prob_otherstate, 0)
            num_tothrow = num_tothrow / prob_otherstate
                                
            ## searching
            optimal_value[0,score_state] = num_tothrow.min()
            optimal_action_index[0,score_state] = num_tothrow.argmin()
    
    else:

        for score_state in range(2,502):            
            ## use matrix operation to search all aiming locations
            
            ## transit to less score state    
            ## s1 = min(score_state-2, 60)
            ## p[z=1]*v[score_state-1] + p[z=2]*v[score_state-2] + ... + p[z=s1]*v[score_state-s1]
            score_max = min(score_state-2, 60)
            score_max_plus1 = score_max + 1 

            ## transit to next state
            num_tothrow = 1.0 
            # tp_nt * v_t <-- save token and make throw
            num_tothrow+= prob_normalscore_1tosmax_dic[score_max].dot(optimal_value[t,score_state-1:score_state-score_max-1:-1])
            # tp_t * v_nt <-- use token 
            num_tothrow+= prob_normalscore_1tosmax_dic_t[score_max].dot(optimal_value[t-1,score_state-1:score_state-score_max-1:-1])
            
            ## probability of transition to state other than s itself
            prob_otherstate = prob_normalscore_1tosmaxsum_dic[score_max] + prob_normalscore_1tosmaxsum_dic_t[score_max]
            
            ## transit to the end of game
            if (score_state == fb.score_DB): ## hit double bull
                prob_otherstate += prob_DB + prob_DB_t
            elif (score_state <= 40 and score_state%2==0): ## hit double
                prob_otherstate += prob_doublescore_dic[score_state]
                prob_otherstate += prob_doublescore_dic_t[score_state]
            else: ## game does not end
                pass
            
            ## expected number of throw for all aiming locations
            prob_otherstate = np.maximum(prob_otherstate, 0)
            num_tothrow = num_tothrow / prob_otherstate
                                
            ## searching
            optimal_value[t,score_state] = num_tothrow.min()
            optimal_action_index[t,score_state] = num_tothrow.argmin()

/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_38840/1551499714.py:70: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_38840/1551499714.py:70: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_38840/1551499714.py:108: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_38840/1551499714.py:108: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


In [4]:
optimal_value

array([[ 0.    ,  0.    ,  2.0893, ..., 14.0691, 14.1144, 14.1292],
       [ 0.    ,  0.    ,  1.    , ..., 12.4836, 12.4423, 12.4665],
       [ 0.    ,  0.    ,  1.    , ..., 12.0083, 11.9611, 11.9643],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  9.8645,  9.8515,  9.8557],
       [ 0.    ,  0.    ,  1.    , ...,  9.4495,  9.4428,  9.4491],
       [ 0.    ,  0.    ,  1.    , ...,  9.    ,  9.    ,  9.    ]])

In [30]:
optimal_action_index

array([[ -1,  -1,  56, ...,  11,  11,  11],
       [ -1,  -1, 752, ...,  11,  11,  11],
       [ -1,  -1, 752, ...,  11,  11,  11],
       ...,
       [ -1,  -1, 752, ...,  11,  11,  11],
       [ -1,  -1, 752, ...,  11,  11,  11],
       [ -1,  -1, 752, ..., 750, 750, 750]], dtype=int32)

In [27]:
optimal_action_index[3][200:240]

array([ 11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11, 418], dtype=int32)

In [28]:
optimal_value[3][200:240]

array([4.4428, 4.4491, 4.4513, 4.4499, 4.4529, 4.4515, 4.4632, 4.4572, 4.4646, 4.4663, 4.4574, 4.5955, 4.4997, 4.5972, 4.6096, 4.5012, 4.6099, 4.612 , 4.6107, 5.1977, 4.628 , 4.6174, 5.2145, 5.215 , 4.6466, 5.2161, 5.2391, 4.6507, 5.2368, 5.2422, 4.6511, 5.2794, 5.2541, 5.2805, 5.3202, 5.2565,
       5.3027, 5.3233, 5.3224, 5.364 ])

## Sanity Checks

Compare to running regularly without tokens and different TP options as sanity check.

In [9]:
# Tokens

[aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore, prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

vals, acts = function_solve_dp.solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore=prob_grid_doublescore, prob_grid_bullscore=prob_grid_bullscore, prob_grid_doublescore_dic=None)

vals
#0., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl


/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp.py:80: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


array([0., 0., 1., ..., 9., 9., 9.])

In [10]:
# No Tokens

[aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore, prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

vals, acts = function_solve_dp.solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore=prob_grid_doublescore, prob_grid_bullscore=prob_grid_bullscore, prob_grid_doublescore_dic=None)

vals
#0., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2., 1., 2

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp.py:80: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


array([ 0.    ,  0.    ,  2.0893, ..., 14.0691, 14.1144, 14.1292])

In [7]:
# Check with turn to make sure TPs are working properly with turns. 

grid_version = 'custom_tokens'
name_pa = 'player{}'.format(10)
postfix=''
gpu_device = None ## Using CPU

function_solve_dp.solve_singlegame(name_pa, data_parameter_dir=data_parameter_dir, grid_version=grid_version, result_dir=result_dir, postfix=postfix, gpu_device=gpu_device)


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
runing solve_dp_turn


/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp.py:347: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp.py:360: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp.py:360: RuntimeWarning: invalid value encountered in divide
  value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()


solve prob_policy_transit in 0.23798584938049316 seconds
solve dp_turn_policyiter in 1.572357177734375 seconds
[0. 0. 1. ... 3. 3. 3.]
dump_pickle to ./result/singlegame_player10_turn_custom_tokens.pkl


In [29]:
a_throw_list[418]

'T19-cc-m'